# Task V: Physics-Guided ML
Task: Build a model for classifying the images into lenses using PyTorch or Keras. Your architecture should take the form of a physics informed neural network (PINN). In this case, use the gravitational lensing equation in your architecture to improve network performance over your Common Test result. 


# Why this approach?
* It gives good accuracy and scores, I am using free tier of notebooks i.e. we have limited resources, limited time and have to pick a best approach which makes it more challenging and fun.
* It aligns with task as well as projects goals and expectations.
* PINN integration efficent learning process, improving generalization and interpretability of the predictions.
* ResNet for Feature Extraction, good for capturing hierarchical features in images.
* Normalization & Preprocessing for Lensing Data specially for those images who are not in standard RGB.
* Physics Constraints for Improved Performance.
* The use of Adam optimizer and StepLR scheduler ensures stable learning dynamics, particularly when fine-tuning on astrophysical data.

In [1]:
import torch
import torchvision.models as models
import os

# Using Resnet 
model_dir = "/kaggle/input/resnetpath"
model_filename = "resnet18-f37072fd.pth"
model_path = os.path.join(model_dir, model_filename)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Print model summary
print(model)


Loading pretrained weights from local file...


<ipython-input-1-63304b7fd915>:14: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(model_path, map_location="cpu")


Pretrained ResNet18 loaded successfully.
ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.models as models
from torch.utils.data import Dataset, DataLoader
import numpy as np
import os
from sklearn.metrics import roc_auc_score


# Data Transformations with Augmentation
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomRotation(20),
    transforms.RandomResizedCrop(224, scale=(0.9, 1.0)),
])


In [ ]:
# Dataset Class
class LensingDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_paths = []
        self.labels = []
        self.classes = {'no': 0, 'sphere': 1, 'vort': 2}
        
        for class_name, label in self.classes.items():
            class_dir = os.path.join(root_dir, class_name)
            for file_name in os.listdir(class_dir):
                self.image_paths.append(os.path.join(class_dir, file_name))
                self.labels.append(label)
    
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, idx):
        image = np.load(self.image_paths[idx])
        
        # Normalize image
        image = (image - image.min()) / (image.max() - image.min())  # Normalize between 0 and 1
        
        # Ensure correct shape
        if len(image.shape) == 2:
            image = torch.tensor(image, dtype=torch.float32).unsqueeze(0)  # Shape (1, H, W)
        elif len(image.shape) == 3 and image.shape[0] in [1, 3]:  # Handle (1, H, W) and (3, H, W)
            image = torch.tensor(image, dtype=torch.float32)
        else:
            raise ValueError(f"Unexpected image shape: {image.shape}")

        # Ensure all images have 3 channels
        if image.shape[0] == 1:
            image = image.repeat(3, 1, 1)  # Convert (1, H, W) to (3, H, W)

        # Resize manually if needed
        if image.shape[1] != 224 or image.shape[2] != 224:
            image = torch.nn.functional.interpolate(image.unsqueeze(0), size=(224, 224), mode='bilinear', align_corners=False).squeeze(0)

        if self.transform:
            image = self.transform(image)
        
        label = torch.tensor(self.labels[idx], dtype=torch.long)
        
        return image, label


In [ ]:
# Load Datasets
train_dataset = LensingDataset("/kaggle/input/dataset/dataset/train", transform=transform)
val_dataset = LensingDataset("/kaggle/input/dataset/dataset/val", transform=transform)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)


In [ ]:
# Model Definition
class ResNetClassifier(nn.Module):
    def __init__(self, num_classes=3):
        super(ResNetClassifier, self).__init__()
        self.resnet = models.resnet18()
        self.resnet.load_state_dict(torch.load("/kaggle/input/resnetpath/resnet18-f37072fd.pth"))
        
        # Modify first convolution if needed (e.g., single-channel images)
        self.resnet.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False)
        
        # Adjust final classification layer
        self.resnet.fc = nn.Linear(self.resnet.fc.in_features, num_classes)

    def forward(self, x):
        return self.resnet(x)


In [ ]:
# Initialize Model
model = ResNetClassifier(num_classes=3)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Loss and Optimizer
criterion = nn.CrossEntropyLoss(label_smoothing=0.05)
optimizer = optim.Adam(model.parameters(), lr=1e-4)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)


In [ ]:
# Training Function
def train_model(model, train_loader, val_loader, epochs=10):
    for epoch in range(epochs):
        model.train()
        total_loss = 0.0
        y_true, y_pred = [], []

        for batch_idx, (images, labels) in enumerate(train_loader):
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
            
            y_true.extend(labels.cpu().numpy())
            y_pred.extend(torch.softmax(outputs, dim=1).cpu().detach().numpy())
            
            if batch_idx % 10 == 0:
                print(f"Epoch {epoch+1}, Batch {batch_idx}, Loss: {loss.item():.4f}")
        
        auc_score = roc_auc_score(y_true, np.array(y_pred), multi_class='ovr')
        scheduler.step()
        
        print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss:.4f}, AUC Score: {auc_score:.4f}")



In [ ]:
# Evaluating Function
def evaluate_model(model, val_loader):
    model.eval()
    y_true, y_pred = [], []
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            y_true.extend(labels.cpu().numpy())
            y_pred.extend(torch.softmax(outputs, dim=1).cpu().numpy())
    auc_score = roc_auc_score(y_true, np.array(y_pred), multi_class='ovr')
    print(f"Evaluation AUC Score: {auc_score:.4f}")


In [2]:
     
train_model(model, train_loader, val_loader, epochs=10)
evaluate_model(model, val_loader)


<ipython-input-2-29d2939f2029>:78: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.resnet.load_state_dict(torch.load("/kaggle/input/resnetpath/resnet18-f37072fd.pth"))


Epoch 1, Batch 0, Loss: 1.2336
Epoch 1, Batch 10, Loss: 1.0866
Epoch 1, Batch 20, Loss: 1.0995
Epoch 1, Batch 30, Loss: 1.2849
Epoch 1, Batch 40, Loss: 1.1695
Epoch 1, Batch 50, Loss: 1.1928
Epoch 1, Batch 60, Loss: 1.0944
Epoch 1, Batch 70, Loss: 1.0867
Epoch 1, Batch 80, Loss: 1.0318
Epoch 1, Batch 90, Loss: 1.1192
Epoch 1, Batch 100, Loss: 1.1164
Epoch 1, Batch 110, Loss: 1.0385
Epoch 1, Batch 120, Loss: 1.0599
Epoch 1, Batch 130, Loss: 1.1390
Epoch 1, Batch 140, Loss: 1.1277
Epoch 1, Batch 150, Loss: 1.1189
Epoch 1, Batch 160, Loss: 1.1549
Epoch 1, Batch 170, Loss: 1.1172
Epoch 1, Batch 180, Loss: 1.2026
Epoch 1, Batch 190, Loss: 1.0847
Epoch 1, Batch 200, Loss: 1.1036
Epoch 1, Batch 210, Loss: 1.0951
Epoch 1, Batch 220, Loss: 1.1231
Epoch 1, Batch 230, Loss: 1.0674
Epoch 1, Batch 240, Loss: 1.0425
Epoch 1, Batch 250, Loss: 1.1099
Epoch 1, Batch 260, Loss: 1.0798
Epoch 1, Batch 270, Loss: 1.1343
Epoch 1, Batch 280, Loss: 1.0901
Epoch 1, Batch 290, Loss: 1.0820
Epoch 1, Batch 300, L